<a href="https://colab.research.google.com/github/educhile1/Challenge_telecom2_ONE/blob/main/Challenge_Telecom2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd

from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay, precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import make_column_transformer


In [58]:
URL = 'https://raw.githubusercontent.com/educhile1/Challenge_telecom2_ONE/bb1797dff49359db1089b33b2241de7131f45bf4/challenge1_datos.json'

datos = pd.read_json(URL)
datos.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,ChargesDaily,ChargesMonthly,ChargesTotal,Churn
0,0002-ORFBO,Female,No,Yes,Yes,9,Yes,No,DSL,No,...,Yes,Yes,No,One year,Yes,Mailed check,2.19,65.6,593.30,No
1,0003-MKNFE,Male,No,No,No,9,Yes,Yes,DSL,No,...,No,No,Yes,Month-to-month,No,Mailed check,2.00,59.9,542.40,No
2,0004-TLHLJ,Male,No,No,No,4,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,2.46,73.9,280.85,Yes
3,0011-IGKFF,Male,Yes,Yes,No,13,Yes,No,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,3.27,98.0,1237.85,Yes
4,0013-EXCHZ,Female,Yes,Yes,No,3,Yes,No,Fiber optic,No,...,Yes,Yes,No,Month-to-month,Yes,Mailed check,2.80,83.9,267.40,Yes


In [59]:
URL2= 'https://raw.githubusercontent.com/educhile1/Challenge_telecom2_ONE/refs/heads/main/cliente_valor_abandona.json'
outliers = pd.read_json(URL2)
## Concatenar ambos DafaFrame v
datos = pd.concat([datos, outliers], axis=0)
## Mezclar los datos
datos = datos.sample(frac=1, random_state=42).reset_index(drop=True)


#Quitar datos no importantes
datos = datos.drop(['CustomerID', 'ChargesDaily'], axis=1)
datos.describe()

,Tenure,ChargesMonthly,ChargesTotal
count,7152.000000,7152.000000,7152.000000
mean,32.851370,65.361955,2346.648399
std,24.684639,30.258020,2314.453371
min,0.000000,18.250000,0.000000
25%,9.000000,36.250000,410.837500
50%,30.000000,70.700000,1424.450000
75%,56.000000,90.400000,3949.575000
max,72.000000,118.750000,8684.800000


In [60]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender            7152 non-null   object 
 1   SeniorCitizen     7152 non-null   object 
 2   Partner           7152 non-null   object 
 3   Dependents        7152 non-null   object 
 4   Tenure            7152 non-null   int64  
 5   PhoneService      7152 non-null   object 
 6   MultipleLines     7152 non-null   object 
 7   InternetService   7152 non-null   object 
 8   OnlineSecurity    7152 non-null   object 
 9   OnlineBackup      7152 non-null   object 
 10  DeviceProtection  7152 non-null   object 
 11  TechSupport       7152 non-null   object 
 12  StreamingTV       7152 non-null   object 
 13  StreamingMovies   7152 non-null   object 
 14  Contract          7152 non-null   object 
 15  PaperlessBilling  7152 non-null   object 
 16  PaymentMethod     7152 non-null   object 


In [61]:
datos.columns

Index(['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'Tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'ChargesMonthly', 'ChargesTotal', 'Churn'],
      dtype='object')

In [62]:
X = datos.drop('Churn', axis=1)
y = datos['Churn']

In [63]:
X_linear = X.copy()
y_linear = y.copy()
X

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,ChargesMonthly,ChargesTotal
0,Female,No,No,No,1,Yes,No,No,No,No,No,No,No,No,Month-to-month,No,Credit card (automatic),19.80,19.80
1,Male,No,Yes,Yes,67,Yes,No,DSL,Yes,Yes,No,Yes,No,No,One year,No,Mailed check,60.05,3994.05
2,Female,No,Yes,No,60,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.80,5985.75
3,Male,No,Yes,No,3,No,No,DSL,Yes,No,No,Yes,No,No,Month-to-month,No,Mailed check,35.10,101.10
4,Female,No,No,No,63,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,Yes,Credit card (automatic),79.85,4861.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7147,Male,No,No,No,46,Yes,No,No,No,No,No,No,No,No,Two year,No,Electronic check,19.95,907.05
7148,Female,No,No,No,1,Yes,No,No,No,No,No,No,No,No,Month-to-month,No,Mailed check,19.15,19.15
7149,Female,No,No,No,4,Yes,Yes,No,No,No,No,No,No,No,One year,No,Electronic check,23.90,97.50
7150,Male,No,No,No,1,Yes,No,No,No,No,No,No,No,No,Month-to-month,No,Mailed check,19.55,19.55


In [64]:
categoricas = ['Gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod']

In [65]:
one_hot = make_column_transformer(
                    (OneHotEncoder(drop='if_binary'), categoricas),
                    remainder='passthrough',
                    sparse_threshold=0,
                    force_int_remainder_cols=False
)

one_hot_linear = make_column_transformer(
                    (OneHotEncoder(drop='first'), categoricas),
                    remainder='passthrough',
                    sparse_threshold=0,
                    force_int_remainder_cols=False
)

label_encoder = LabelEncoder()

In [66]:
X = one_hot.fit_transform(X)
X_linear = one_hot_linear.fit_transform(X_linear)

y = label_encoder.fit_transform(y)
y_linear = label_encoder.fit_transform(y_linear)

In [70]:
columnas_linear = one_hot_linear.get_feature_names_out()

columnas_encoded_linear = []
for columna in columnas_linear:
    columna = columna.split('__')[1]
    columnas_encoded_linear.append(columna)

df_linear = pd.DataFrame(X_linear, columns=columnas_encoded_linear)
df_y_linear = pd.DataFrame(y_linear, columns=['Churn'])
df_linear = pd.concat([df_linear, df_y_linear], axis=1)
df_linear.head()

,Gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_Yes,OnlineBackup_Yes,...,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Tenure,ChargesMonthly,ChargesTotal,Churn
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,19.80,19.80,0
1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,67.0,60.05,3994.05,0
2,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,60.0,99.80,5985.75,0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,35.10,101.10,0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,63.0,79.85,4861.45,0
